In [51]:
import numpy as np
import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm
from qutip import tensor, destroy, qeye, num
# Define system parameters (in GHz)
EC1 = 0.21522476919932843  # Charging energy of qubit 1 (GHz)
EC2 = 0.21522476919932843  # Charging energy of qubit 2 (GHz)
EJ1 = 15.857238090360163   # Josephson energy of qubit 1 (GHz)
EJ2 = 14.469215483725697   # Josephson energy of qubit 2 (GHz)
freq_resonator = 8.5  # Resonator frequency (GHz)
g1 = 0.05  # Coupling strength between qubit 1 and resonator (GHz)
g2 = 0.05  # Coupling strength between qubit 2 and resonator (GHz)
conversion_constant1 = -1j*(EJ1/8/EC1)**(1/4)*1/np.sqrt(2)
conversion_constant2 = -1j*(EJ2/8/EC2)**(1/4)*1/np.sqrt(2)
g1_k = g1/conversion_constant1
g2_k = g2/conversion_constant2

ncutoff = 9  # Charge basis cutoff for transmons
truncation_level = 9  # Truncate the Hilbert space for easier diagonalization

# Define Transmon qubits and the resonator
qubit1 = scq.Transmon(EJ=EJ1, EC=EC1, ng=0.0, ncut=ncutoff, truncated_dim=truncation_level)
qubit2 = scq.Transmon(EJ=EJ2, EC=EC2, ng=0.0, ncut=ncutoff, truncated_dim=truncation_level)
resonator = scq.Oscillator(E_osc=freq_resonator, truncated_dim=truncation_level)

# Define the Hilbert space
hilbertspace = HilbertSpace([qubit1, qubit2, resonator])

hilbertspace.add_interaction(
    g = g1_k,
    op1 = qubit1.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True
)

hilbertspace.add_interaction(
    g = g2_k,
    op1 = qubit2.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True
)
hilbertspace.generate_lookup()

# Calculate energy differences for qubit states
E_00 = hilbertspace.energy_by_bare_index((0,0,0), subtract_ground = True)  # Ground state (|00⟩)
E_01 = hilbertspace.energy_by_bare_index((0,1,0), subtract_ground = True)  # Qubit 1 in ground state, qubit 2 in excited state (|01⟩)
E_10 = hilbertspace.energy_by_bare_index((1,0,0), subtract_ground = True)  # Qubit 1 in excited state, qubit 2 in ground state (|10⟩)
E_11 = hilbertspace.energy_by_bare_index((1,1,0), subtract_ground = True)  # Both qubits in excited state (|11⟩)

E_02 = hilbertspace.energy_by_bare_index((0,2,0), subtract_ground = True)
E_20 = hilbertspace.energy_by_bare_index((2,0,0), subtract_ground = True)

# Calculate the ZZ interaction
ZZ_interaction = (E_11 - E_10 - E_01)

# Display the results
print(f"Static ZZ interaction: {ZZ_interaction} ")  # Convert from GHz to MHz

Static ZZ interaction: 0.00011332525768548152 


In [52]:
def ZZ(J, d1, d2, D, W):
    zz = J**2 * (d1 + d2) / (D + d1) / (D - d2)
    
    return zz

def calculate_J(g1, g2, omega1, omega2, omega_r):
    numerator = g1 * g2 * (omega1 + omega2 - 2 * omega_r)
    denominator = 2 * (omega1 - omega_r) * (omega2 - omega_r)
    J = numerator / denominator
    return J

In [53]:
a1 = qubit1.anharmonicity() #E_20 - 2*E_10 
a2 = qubit2.anharmonicity() 
w1 = qubit1.E01()
w2 = qubit2.E01()
D = w1 - w2
J = calculate_J(g1, g2, w1, w2, freq_resonator)
zz = ZZ(J, a1, a2, D, 0)
print(zz)

8.65732757680709e-05


In [39]:
J

-0.000378874856494593

In [40]:
g1, g2, w1, w2, freq_resonator

(0.05, 0.05, 5.000000000263521, 4.799999999999988, 11.5)

In [41]:
def ZZ(J, d1, d2, D, W):
    zz = J**2 * (d1 + d2) / (D + d1) / (D - d2)
    
    return zz

def calculate_J(g1, g2, omega1, omega2, omega_r):
    numerator = g1 * g2 * (omega1 + omega2 - 2 * omega_r)
    denominator = 2 * (omega1 - omega_r) * (omega2 - omega_r)
    J = numerator / denominator
    return J